In [8]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import json

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "inprompt"

In [9]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [10]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'remember information about each person and then answer the question in POLISH', 'input': ['Abdon ma czarne oczy, średniej długości włosy i pracuje jako prawnik, a na śniadanie najbardziej lubi jeść owsiankę', 'Abel ma czarne oczy, długie włosy i pracuje jako lekarz, a na śniadanie najbardziej lubi jeść kanapki z serem', 'Abelard ma zielone oczy, długie włosy i pracuje jako murarz, a na śniadanie najbardziej lubi jeść kanapki z serem', 'Abraham ma zielone oczy, średniej długości włosy i pracuje jako architekt, a na śniadanie najbardziej lubi jeść jogurt z owocami', 'Achilles ma czarne oczy, średniej długości włosy i pracuje jako fryzjer, a na śniadanie najbardziej lubi jeść kurczaka', 'Adam ma szare oczy, średniej długości włosy i pracuje jako nauczyciel, a na śniadanie najbardziej lubi jeść kurczaka', 'Adelard ma szare oczy, krótkie włosy i pracuje jako prawnik, a na śniadanie najbardziej lubi jeść jogurt z owocami', 'Adnan ma czarne oczy, średn

In [11]:
chat = ChatOpenAI()

# znalezienie imienia występującego w pytaniu
name_response = chat.invoke([
    SystemMessage(f"Znajdź imię w pytaniu. Zwróć tylko to imię i nic więcej. Imię:"),
    HumanMessage(f"Pytanie: {response['question']}")
])

# znalezienie zdań, które zawierają imię z pytania
name_info = [s for s in response['input'] if s.find(name_response.content) != -1]

chat = ChatOpenAI()
# odpowiedź na pytanie na podstawie wybranego kontekstu
question_response = chat.invoke([
    SystemMessage(f"""Based on the information in the context, answer the question. If the information from the context is not sufficient, answer 'don't know'
                  Context:###
                  {name_info}
                  \###"""),
    HumanMessage(f"Pytanie: {response['question']}")
])

# Tworzenie słownika z właściwością "answer"
answer_dict = {"answer": question_response.content}
# Konwersja słownika na format JSON
answer_json = json.dumps(answer_dict)

print(answer_json)

{"answer": "Ezaw ma niebieskie oczy."}


In [12]:
# wysłanie odpowiedzi

response = send_answer(token, answer_json)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
